In [1]:
!which python

/Users/ahemf/anaconda3/envs/hybrid-recsys/bin/python


In [2]:
!pip install spacy > /dev/null
!python -m spacy download en > /dev/null
!conda install -y xgboost
!pip install flair  


In [1]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

import time
import matplotlib.pyplot as plt


import nltk
from nltk import word_tokenize
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
stopwords_list.extend(["ul","li","br","hr","h1","h2","h3","h4","h5","h6"])
stopwords_list = set(stopwords_list)

from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, HTML
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format='retina'

import re
from joblib import Parallel, delayed

import more_itertools
from more_itertools import flatten
import ast

import gc

import sys
import os
sys.path.append(os.getcwd())
from importlib import reload

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
from more_itertools import flatten
import dill
from collections import Counter

import itertools
from tqdm import tqdm_notebook

from numpy import dot
from numpy.linalg import norm
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.neighbors import NearestNeighbors
import nmslib
import time
from tqdm import tqdm_notebook


def unit_length(a, axis=0):
    return a/np.expand_dims(norm(a, axis=axis), axis=axis)

[nltk_data] Downloading package wordnet to /Users/ahemf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ahemf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ahemf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("training.csv")
df_test = pd.read_csv("public_test_features.csv")

In [4]:
df.human_tag.value_counts()


0    53375
1     9759
Name: human_tag, dtype: int64

In [5]:
df.head()

,ID,doc_id,text,date,star_rating,title,human_tag
0,47490,15808037321,"I ordered a sample of the Dietspotlight Burn, ...",6/25/2018 17:51,1,DO NOT BUY!,0
1,16127,16042300811,This coffee tasts terrible as if it got burnt ...,2/8/2018 15:59,2,Coffee not good,0
2,51499,16246716471,I've been buying lightly salted Planters cashe...,3/22/2018 17:53,2,"Poor Quality - Burnt, Shriveled Nuts With Blac...",0
3,36725,14460351031,This product is great in so many ways. It goes...,12/7/2017 8:49,4,"Very lovey product, good sunscreen, but strong...",0
4,49041,15509997211,"My skin did not agree with this product, it wo...",3/21/2018 13:51,1,Not for everyone. Reactions can be harsh.,1


In [3]:
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence, BytePairEmbeddings, StackedEmbeddings

glove_embedding = WordEmbeddings('glove')
crawl_embedding = WordEmbeddings('crawl')
flair_embedding_forward = FlairEmbeddings('news-forward-fast')
flair_embedding_backward = FlairEmbeddings('news-backward-fast')
bp_embedding = BytePairEmbeddings('en')




In [20]:
# initialize the document embeddings, mode = mean
document_embeddings = DocumentPoolEmbeddings([bp_embedding,glove_embedding])

In [4]:
document_embeddings = DocumentPoolEmbeddings([bp_embedding,crawl_embedding])

# document_embeddings = DocumentPoolEmbeddings([flair_embedding_forward])

sentence = Sentence('The grass is green . And the sky is blue .')

# embed the sentence with our document embedding
document_embeddings.embed(sentence)

# now check out the embedded sentence.
a = sentence.get_embedding()
a.detach().numpy().shape

(1024,)

# Build Sentence

In [4]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    if type(text) is pd.core.series.Series or type(text) is str:
        text = text.replace("'", " ").replace('"', " ")
        clean = re.compile('<.*?>')
        return re.sub(clean, ' ', text)
    return text


def clean_text(text):
    EMPTY = ''
    text = text.replace("\n", " ").replace("(", " ").replace(")", " ").replace("\r", " ").replace("\t", " ").lower()
    text = re.sub('<pre><code>.*?</code></pre>', EMPTY, text)
    text = re.sub('<code>.*?</code>', EMPTY, text)

    def replace_link(match):
        EMPTY = ''
        return EMPTY if re.match('[a-z]+://', match.group(1)) else match.group(1)

    text = re.sub('<a[^>]+>(.*)</a>', replace_link, text)
    return text



In [6]:
def get_sentence_vector(text):
    sentence = Sentence(text)

    # embed the sentence with our document embedding
    document_embeddings.embed(sentence)

    # now check out the embedded sentence.
    a = sentence.get_embedding()
    return a.detach().numpy()


In [7]:
df[['star_rating','human_tag']].corr()

,star_rating,human_tag
star_rating,1.000000,-0.269447
human_tag,-0.269447,1.000000


In [9]:
def build_features(df):
    df['sentence'] = df['title'].fillna('') + ' ' + df['text'].fillna('')
    df["sentence"] = Parallel(n_jobs=1, backend="loky")(delayed(lambda x:remove_html_tags(clean_text(x)))(x) for x in tqdm_notebook(df["sentence"].values))

    df_vecs = pd.DataFrame(np.vstack(Parallel(n_jobs=1, backend="loky")(delayed(get_sentence_vector)(x) for x in tqdm_notebook(df['sentence'].values))))
    return pd.concat((df[["star_rating"]],df_vecs),axis=1)
    
    
    

In [10]:
X_train = build_features(df)


KeyboardInterrupt: 

In [33]:
X_test = build_features(df_test)

In [34]:
y_train = df['human_tag'].values

# XGBoost Model

In [67]:

from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

xgb = XGBClassifier(max_depth=5, learning_rate=0.3, n_estimators=50, n_jobs=-1,scale_pos_weight=4, base_score=0.5,
                   booster='gbtree', tree_method='hist',gamma=16, reg_alpha=0.1, reg_lambda=1)

cv_results = cross_validate(xgb, X_train, y_train, cv=4, scoring=['accuracy','f1'],return_train_score=True)


In [68]:
cv_results

{'fit_time': array([29.90423512, 39.90726018, 34.30079007, 29.92018104]),
 'score_time': array([0.88347006, 0.83514977, 0.53931999, 0.4690361 ]),
 'test_accuracy': array([0.83141156, 0.8348961 , 0.83280537, 0.83043974]),
 'train_accuracy': array([0.89151003, 0.8918057 , 0.89269271, 0.89280284]),
 'test_f1': array([0.54254771, 0.56098383, 0.55096137, 0.56217277]),
 'train_f1': array([0.71570092, 0.71678921, 0.71880015, 0.71803133])}

In [48]:
cv_results

{'fit_time': array([42.48375511, 44.680511  , 43.27564812, 42.53770638]),
 'score_time': array([0.49297404, 0.59838581, 0.72580409, 0.52426481]),
 'test_accuracy': array([0.82254181, 0.82152813, 0.82380892, 0.81890762]),
 'train_accuracy': array([0.84956705, 0.84523759, 0.84669483, 0.84714479]),
 'test_f1': array([0.54358807, 0.55560814, 0.55454109, 0.55357701]),
 'train_f1': array([0.62170057, 0.61540884, 0.61872998, 0.61679373])}

In [69]:
xgb = XGBClassifier(max_depth=5, learning_rate=0.3, n_estimators=50, n_jobs=-1,scale_pos_weight=4, base_score=0.5,
                   booster='gbtree', tree_method='hist',gamma=16, reg_alpha=0.1, reg_lambda=1)

xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=16,
              learning_rate=0.3, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.1, reg_lambda=1, scale_pos_weight=4, seed=None,
              silent=None, subsample=1, tree_method='hist', verbosity=1)

In [70]:
y_pred = xgb.predict(X_test)

In [71]:
y_pred

array([0, 0, 1, ..., 0, 0, 0])

In [72]:
df_sub = df_test[["ID"]]
df_sub['human_tag'] = y_pred
df_sub.to_csv("submission.csv",index=False)

In [73]:
df_sub.head()

,ID,human_tag
0,62199,0
1,76123,0
2,78742,1
3,64010,0
4,17058,0


In [76]:
def feature_importance(model,features):
    """

    :param model: Model object which has `feature_importances_`
    :param features: features/columns that were given to model, these must be in same order as given to model
    :return: DataFrame with sorted feature importances
    """
    if hasattr(model, 'feature_importances_'):
        fi=model.feature_importances_
    elif hasattr(model, 'coef_'):
        fi = model.coef_
    else:
        raise AttributeError('No attribute: feature_importances_ or  coef_')
    df_i=pd.DataFrame({"feature":features,"importance":fi})
    df_i["importance"] = df_i["importance"]*100
    return df_i.sort_values("importance",ascending=False)

feature_importance(xgb, X_train.columns)

,feature,importance
0,star_rating,8.790469
206,205,3.552739
185,184,2.161792
337,336,1.999914
368,367,1.179129
...,...,...
22,21,0.000000
238,237,0.000000
283,282,0.000000
91,90,0.000000


# Flair Text Classification
https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f

https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md


In [5]:
df['sentence'] = df['title'].fillna('') + ' ' + df['text'].fillna('')
df_test['sentence'] = df_test['title'].fillna('') + ' ' + df_test['text'].fillna('')

df["flair_text"] = "__label__" + df["human_tag"].astype(str) +" " + df['sentence'].fillna("") + " star_" + df['star_rating'].astype(str).fillna("")
df["flair_text"] = Parallel(n_jobs=1, backend="loky")(delayed(lambda x:remove_html_tags(clean_text(x)))(x) for x in tqdm_notebook(df["flair_text"].values))

df_test["flair_text"] = df_test['sentence'].fillna("") + " star_" + df_test['star_rating'].astype(str).fillna("")
df_test["flair_text"] = Parallel(n_jobs=1, backend="loky")(delayed(lambda x:remove_html_tags(clean_text(x)))(x) for x in tqdm_notebook(df_test["flair_text"].values))



In [110]:
df['flair_text'].head()
np.sum(df['flair_text'].isna())

0    __label__0 do not buy! i ordered a sample of t...
1    __label__0 coffee not good this coffee tasts t...
2    __label__0 poor quality - burnt, shriveled nut...
3    __label__0 very lovey product, good sunscreen,...
4    __label__1 not for everyone. reactions can be ...
Name: flair_text, dtype: object

0

In [6]:
from sklearn.model_selection import train_test_split
from pathlib import Path
data_folder = './flair_data/'
path = Path(data_folder)


In [13]:
df_flair = df[['flair_text','human_tag']]
neg_samples = df_flair[df_flair['human_tag']==0]
pos_samples = df_flair[df_flair['human_tag']==1]

df_flair = pd.concat((neg_samples.sample(frac=1.0),pos_samples.sample(frac=1.0),
                      pos_samples.sample(frac=1.0),pos_samples.sample(frac=1.0),
                      pos_samples.sample(frac=1.0),pos_samples.sample(frac=0.75),pos_samples.sample(frac=0.75)))

df_flair = df_flair.sample(frac=1.0)
df_flair.human_tag.value_counts()


1    53674
0    53375
Name: human_tag, dtype: int64

In [14]:



FlairTrain, FlairTest = train_test_split(df_flair[['flair_text']],test_size=0.2)
FlairTrain, FlairDev = train_test_split(FlairTrain,test_size=0.2)


FlairTrain.to_csv(str(path.absolute())+"/train.csv",sep='\t', index = False, header = False)
FlairTest.to_csv(str(path.absolute())+"/test.csv",sep='\t', index = False, header = False)
FlairDev.to_csv(str(path.absolute())+"/dev.csv",sep='\t', index = False, header = False)

df_flair[['flair_text']].to_csv(str(path.absolute())+"/train_full.csv",sep='\t', index = False, header = False)


In [ ]:

from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus, ClassificationCorpus

from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path


corpus: Corpus = ClassificationCorpus(path,
                                      test_file='test.csv',
                                      dev_file='dev.csv',
                                      train_file='train_full.csv')
    
word_embeddings = [glove_embedding, bp_embedding]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=16, rnn_layers=1, reproject_words=True, reproject_words_dimension=128, 
                      bidirectional=True, dropout=0.1, word_dropout=0.1, rnn_type="GRU")
    
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=5, learning_rate = 0.1, mini_batch_size = 128,
              num_workers = 2, anneal_factor = 0.25,patience = 2,)


2019-11-01 20:56:42,871 Reading data from flair_data
2019-11-01 20:56:42,872 Train: flair_data/train_full.csv
2019-11-01 20:56:42,875 Dev: flair_data/dev.csv
2019-11-01 20:56:42,876 Test: flair_data/test.csv
2019-11-01 20:56:46,891 Computing label dictionary. Progress:


100%|██████████| 107049/107049 [02:03<00:00, 869.50it/s] 

2019-11-01 20:58:50,562 [b'1', b'0']


2019-11-01 20:58:50,968 ----------------------------------------------------------------------------------------------------
2019-11-01 20:58:50,969 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): BytePairEmbeddings(model=1-bpe-en-100000-50)
    )
    (word_reprojection_map): Linear(in_features=200, out_features=128, bias=True)
    (rnn): GRU(128, 16, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (word_dropout): WordDropout(p=0.1)
  )
  (decoder): Linear(in_features=64, out_features=2, bias=True)
  (loss_function): CrossEntropyLoss()
)"
2019-11-01 20:58:50,970 ----------------------------------------------------------------------------------------------------
2019-11-01 20:58:50,973 Corpus: "Corpus: 107049 train + 17128 dev + 21410 test sentences"
2019-11-01 20:58:50,974 ----------------------------------------

In [21]:

# classifier = TextClassifier.load_from_file('./best-model.pt')


[Sentence: "Hi. Yes mum, I will..." - 5 Tokens]

1 ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_score', '_value', 'score', 'to_dict', 'value']


In [24]:
test_sentences = list(map(Sentence,df_test['flair_text'].values))

In [ ]:
test_sentences = list(map(Sentence,df_test['flair_text'].values))
_ = classifier.predict(test_sentences)
y_preds = list(map(lambda s:s.labels[0].value,test_sentences))


In [23]:
df_sub = df_test[["ID"]]
df_sub['human_tag'] = y_preds
df_sub.to_csv("submission.csv",index=False)